## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-13-04-31 +0000,bbc,"Reggae legend Jimmy Cliff dies, aged 81",https://www.bbc.com/news/articles/cj4qdren425o...
1,2025-11-24-13-04-02 +0000,nyt,U.S. and Ukraine Hail ‘Meaningful Progress’ on...,https://www.nytimes.com/2025/11/24/world/europ...
2,2025-11-24-12-53-11 +0000,nyt,How One German Toymaker Made Money Despite U.S...,https://www.nytimes.com/2025/11/24/business/ge...
3,2025-11-24-12-52-25 +0000,nypost,Freed Israeli hostage details Hamas sexual abu...,https://nypost.com/2025/11/24/world-news/israe...
4,2025-11-24-12-52-09 +0000,startribune,"Accumulating snow in forecast for Twin Cities,...",https://www.startribune.com/accumulating-snow-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,ukraine,21
392,trump,18
11,plan,16
10,peace,14
361,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...,111
95,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,99
94,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,83
185,2025-11-23-17-38-05 +0000,nypost,Zelensky thanks Trump for support after critic...,https://nypost.com/2025/11/23/world-news/zelen...,80
217,2025-11-23-13-39-57 +0000,nypost,"Ukraine peace plan was authored by US, Rubio s...",https://nypost.com/2025/11/23/world-news/ukrai...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,111,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...
119,44,2025-11-24-02-21-21 +0000,nypost,New ‘NYC Noise’ app allows New Yorkers to poli...,https://nypost.com/2025/11/23/us-news/new-nyc-...
173,37,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
26,33,2025-11-24-11-39-26 +0000,nypost,Erika Kirk reveals she prayed she was pregnant...,https://nypost.com/2025/11/24/us-news/erika-ki...
134,30,2025-11-24-00-15-23 +0000,nypost,Florida homeowner turns tables on home invader...,https://nypost.com/2025/11/23/us-news/homeowne...
148,28,2025-11-23-22-31-51 +0000,nypost,Canada’s prime minister says trade talks with ...,https://nypost.com/2025/11/23/business/canadas...
24,25,2025-11-24-11-45-12 +0000,nyt,"Jamil Abdullah Al-Amin, Black Power Activist K...",https://www.nytimes.com/2025/11/23/us/h-rap-br...
127,25,2025-11-24-00-53-47 +0000,nypost,"NYC mob pummels couple, sets vehicle on fire a...",https://nypost.com/2025/11/23/us-news/nyc-mob-...
25,24,2025-11-24-11-42-00 +0000,wsj,Health Care Roundup: Market Talk,https://www.wsj.com/business/health-care-round...
34,24,2025-11-24-11-00-00 +0000,nypost,Long Island pot shops sell most expensive cann...,https://nypost.com/2025/11/24/us-news/long-isl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
